# 基于PaddleHub的18IG队员人物素描
该项目应用U2Net预训练模型，搭建一个可以end2end生成人体面部素描的应用程序。本文将详细介绍load预训练模型、图片裁剪、缝合的各种细节，并有U2Net模型的深度讲解。欢迎大家一键star、一键fork。

关于数据准备，由于本人为iglol粉丝，将ig的春季赛定妆照应用到了该程序中，效果十分可观。

这里是github项目，欢迎star fork！[github链接](https://github.com/Stuyxr/invictus-Gaming-members-portrait-generation) 


# 一、效果展示
这里是bilibili项目展示， \
[AI创造营 X IG](https://www.bilibili.com/video/BV1Zh411Q7Nr)
<iframe style="width:98%;height: 450px;" src="//player.bilibili.com/player.html?aid=544408715&bvid=BV1Zh411Q7Nr&cid=303813834&page=1" scrolling="no" border="0" frameborder="no" framespacing="0" allowfullscreen="true"> </iframe>

IG啊IG，今年你还有机会吗\
效果图：\
第一排TheShy，Duke，Ning，Rookie，JackeyLove\
第三排中间三个Baolan~\
是我最喜欢的18極了\
source images:\
![](https://ai-studio-static-online.cdn.bcebos.com/a9c6f640a5ae4f89a4ffa19bb55754d213e659e6ec5c4fca94d284373e33868f)
generated images:\
![](https://ai-studio-static-online.cdn.bcebos.com/719712e91eae44269498c55ef3786a93de1882155c284b72b6a9f57c056262c1)
# 二、实现思路
安装paddlehub -> 数据准备 -> 创建模型 -> 加载模型 -> 推断(inference)

# 三、U2Net模型概述
在这一部分将详细介绍U2Net网络。 

U2Net最初用于显著目标检测(Salient Object Detection, SOD) ，目的是分割出图像中最具吸引力的目标。不同于图像识别，SOD更注重局部细节信息和全局对比信息，而不是深层语义信息，因此，主要的研究方向在于多层次与多尺度特征提取上。 

而我们要做的portrait其实也是一种分割结果，无非就是一些特征分割出来，最终看起来像是人物肖像。按照这个思路其实U2net可以完全胜任这个问题。 
因此U2Net_portrait模型就是U2Net模型用portrait dataset训练出来的结果。 

那么U2Net是怎么做显著目标检测的？ 

对于SOD和其他分割任务来说，局部和全局上下文信息都非常重要。但是现有网络大部分是利用的一些经典image classification网络（如VGG，AlexNet等）提取深度特征，但这些缺少局部细节和全局对比的信息，而更多的是图像的语义信息，而SOD正是需要这种局部和全局间的关系等信息。

因此，U2Net利用UNet的结构，并融合了FCN的一些特征融合技巧，采用了如下的Residual U-blocks。

![](https://ai-studio-static-online.cdn.bcebos.com/513d1d14307f4d9a9b94929923c192da6afe846bf91847d69478c4d42e9fffcb)

网络结构以及把一整个u2net完整的表示出来了，作者提出了一种名为RSU的新模块。对于每个RSU本身就是一个小号的Unet，最后所有的RSU用一种类似FPN的结构连接在一起。类似down-top top-down。通过这种方式来增加多尺度能力。获得了极为优秀的分割结果。

下图右侧为每一个block的结构，和resnet block有一些类似。

![](https://ai-studio-static-online.cdn.bcebos.com/b409e15600e84c358e8728fd38e9d04e4dc02ecc54a94934a0afbc6c6d3358b3)

# 四、代码实现
## 安装paddlehub
安装paddlehub，这一步很简单，直接pip install即可。

In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!pip install paddlehub==2.0.0b2

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 204kB 13.0MB/s eta 0:00:01
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


## 数据准备
在项目文件夹中有source image 

准备了几张照片（19春季赛的定妆照 

## 导入U2Net_Portrait模型
项目采用了U2Net_Portrait网络。从PaddleHub里导入了预训练模型。如果想要使用GPU需要设置一下`os.environ["CUDA_VISIBLE_DEVICES"]`为"0"。当然这里可以不使用GPU。\
Paddlehub集成了很多预训练模型，我们可以从paadlehub里直接将U2Net_Portrait预训练网络加载到内存中。\
`hub.Module()`是创建模型，模型使用的参数根据其构造参数`name`来确定。这里`name='U2Net_Portrait'`就是在指定模型参数。\
通过调用模型的`Portrait_GEN`方法，获取模型输出的结果。`Portrait_GEN`方法的输入参数有`images`和`output_dir`，顾名思义是输入图片和输出路径。最后用`opencv`将图片保存即可。

In [ ]:
import paddlehub as hub
import os
import cv2

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# 模型加载
# use_gpu：是否使用GPU进行预测
for f in os.listdir('./18ji'):
    model = hub.Module(name='U2Net_Portrait')
    # img = cv2.imread(os.path.join('./18ji', f))
    # result = model.Cartoon_GEN(images=[img], output_dir='./output_frames')
    result = model.Portrait_GEN(images=[img], output_dir='./output_frames2')
    # print(result[0].shape)
    cv2.imwrite(f'./18ji_output/{f}', result[0])

146.0


![](https://ai-studio-static-online.cdn.bcebos.com/2625bf36fb154b588f17b88c40c3dff29a8c67992c784670afdd87ebf504ff56)
![](https://ai-studio-static-online.cdn.bcebos.com/ed638b05becc47e4b2740cdb148156fbe1aae83b11ff4100bdda333e0f9c15bf)
![](https://ai-studio-static-online.cdn.bcebos.com/556c195e315549dbbd519a00a88b3fa012cf9de569f04d87984e432d5d5ba6d4)
![](https://ai-studio-static-online.cdn.bcebos.com/22ea6474e5c14c8dbed57467e51b49c06f78b54e951d494aa2614d56d0aa433d)
![](https://ai-studio-static-online.cdn.bcebos.com/8a2e7fee27c9477f942f1cf6486511bd554634e48a054253b5837c226f9c9d34)
![](https://ai-studio-static-online.cdn.bcebos.com/92979f19e44e4fddb498b9cd8767b58150fca99a97544f5bae22f714eb82438d)
![](https://ai-studio-static-online.cdn.bcebos.com/aade7d1b9f19485e87c70c583f7199ea836eabcd81b54255a1d3ebb3986ea4ff)

（呜呜呜shy哥好帅我要放两张）


## 合成为一张图
这段代码将生成图片resize到128x128并拼在一起，用numpy数组进行赋值操作

In [16]:
import cv2
import numpy as np
import os

root = './18ji_output'
w, h = 128, 128
cnt_h = 4
cnt_w = 5
res = np.zeros((h * cnt_h, w * cnt_w, 3))
now_h = 0
now_w = 0
for path in os.listdir(root):
    path = os.path.join(root, path)
    img = cv2.resize(cv2.imread(path), (w, h))
    # cv2.imwrite('f.png', img)
    # print(now_h * h, (now_h + 1) * h)
    res[now_h * h: (now_h + 1) * h, now_w * w: (now_w + 1) * w, :] = img
    now_w += 1
    if now_w == cnt_w:
        now_w = 0
        now_h += 1

cv2.imwrite('res.png', res)

True

![](https://ai-studio-static-online.cdn.bcebos.com/719712e91eae44269498c55ef3786a93de1882155c284b72b6a9f57c056262c1)


# 关于作者
哈尔滨工业大学 计算机科学与技术专业 大四在读\
感兴趣的方向为：计算机视觉、自然语言处理\
AIstudio主页：314920465\
Github主页：stuyxr\
欢迎大家有问题留言交流学习，共同进步成长